In [ ]:
import ast
str = """
import sys
import os
def f1(l):
    l.reverse()
    reverse(l)
    l.sort()
    sorted(l)
    len(l)
    for x in l:
        pass
    return(l)"""
tree = ast.parse(str)
tree
exec(compile(tree, filename="<ast>", mode = "exec"))

In [ ]:
import ast
from collections import deque


class FuncCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()

    @property
    def name(self):
        #print(self._name)
        return '.'.join(self._name)


    @name.deleter
    def name(self):
        self._name.clear()

    # Updating to only show obj for ids
    def visit_Name(self, node):
        self._name.appendleft(node.id)


    def visit_Attribute(self, node):
        try:
            self._name.appendleft(node.attr)
            # hacking for demonstration list of functions
            #self._name.appendleft(node.value.id)
            self._name.appendleft("")

            #print(node.value.id)
        except AttributeError:
            self.generic_visit(node)


def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FuncCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append(callvisitor.name)
    result = {}
    for item in func_calls:
      result[item] = True
    #return sorted(list(set(func_calls)))
    return result

In [ ]:
# Collect all statements
def getAllStat(tree):
  stat = {}
  for node in ast.walk(tree):
    if isinstance(node, ast.While):
      stat["While"] = True
    elif isinstance(node, ast.For):
      stat["For"] = True
    elif isinstance(node, ast.Return):
      stat["Return"] = True
    elif isinstance(node, ast.If):
      stat["If"] = True
    #else:
    #  print(node)

  return stat

In [ ]:
# Collect all expressions
def getAllExpr(tree):
  expr = []
  for node in ast.walk(tree):
    if isinstance(node, ast.Add):
      expr.append("+")
    if isinstance(node, ast.Eq):
      expr.append("==")
    if isinstance(node, ast.Sub):
      expr.append("-")
    if isinstance(node, ast.Mult):
      expr.append("*")
    if isinstance(node, ast.Div):
      expr.append("/")
    if isinstance(node, ast.Lt):
      expr.append("<")
    if isinstance(node, ast.Gt):
      expr.append(">")

  result = {}
  for item in expr:
    result[item] = True
  return result

In [ ]:
def getAllImports(a):
	"""Gather all imported module names"""
	if not isinstance(a, ast.AST):
		return set()
	imports = set()
	for child in ast.walk(a):
		if type(child) == ast.Import:
			for alias in child.names:
					imports.add(alias.asname if alias.asname != None else alias.name)
		elif type(child) == ast.ImportFrom:
				for alias in child.names: # these are all functions
						imports.add(alias.asname if alias.asname != None else alias.name)
	result = {}
	for item in imports:
	  result[item] = True
	return result

getAllImports(tree)

{'os': True, 'sys': True}

In [ ]:
def code_features(tree):
  result = {
        "statements":[],
        "functions":[],
        "imports":[],
        "expressions":[]
    }
  result["imports"] = getAllImports(tree)
  result["expressions"] =getAllExpr(tree)
  result["functions"] = get_func_calls(tree)
  result["statements"] = getAllStat(tree)
  return result

code_features(tree)

{'expressions': {},
 'functions': {'.reverse': True,
  '.sort': True,
  'len': True,
  'reverse': True,
  'sorted': True},
 'imports': {'os': True, 'sys': True},
 'statements': {'For': True, 'Return': True}}

In [ ]:
# Some sample solutions data
"""
{ problemKey1: {
     solutionKey1: {},
     solutionKey2: {}
  },
  problemKey2: {
     userKey1: {},
     userKey2: {}
  }
}

"""

test_dict = {'-LEgrNOmnWsFkr6nFHwO': {'0uB55jXtOnQUtFffPYK1aGLiZO33': '# return the values of a dictionary d as a list\ndef listValues(d):\n    l = []\n    for v in d.values():\n        l.append(v)\n    return l\n\n', '3MBpk3DgmXTJeKqVTEq3vi4bpML2': '# return the values of a dictionary d as a list\ndef listValues(d):\n    return list(d.values())\n\n', '9D72RQCGQadyNE83vqvTNukk7Fz2': '# return the values of a dictionary d as a list\n# def listValues(d)\ndef listValues(d):\n  return list(d.values())\n  \n\n', 'TOT2Pe5KKIe8QufgPxM2S22VqHv1': '# return the values of a dictionary d as a list\n# def listValues(d)\ndef listValues(d):\n  return(list(d.values()))\n\n', 'eVJVC9kde3QSiXAP989kivD9SZn2': '# return the values of a dictionary d as a list\ndef listValues(d):\n    result = []\n    for key in d: \n        result.append(d[key])    \n    return result\n    \n\n'}}

url = "https://s3-ap-southeast-1.amazonaws.com/alset-public/example_solutions.json"
import urllib.request
import json
req = urllib.request.Request(url)

##parsing response
r = urllib.request.urlopen(req).read()
student_solutions = json.loads(r.decode('utf-8'))

for problemKey in student_solutions.keys():
  #print(problemKey)
  for userKey in student_solutions[problemKey].keys():
    #print("   ",userKey)
    #print(student_solutions[problemKey][userKey])
    pass

In [ ]:

## final code for recos
def getSolutionFeatures(student_solutions):
  #Try listing funcitons from the student_solutions

  problem_key_dic = {}
  user_count_dic ={}
  for prob_count,problemKey in enumerate(student_solutions.keys()):
    dic ={'imports':{},'statements':{},'expressions':{},'functions':{}}

    for user_count,userKey in enumerate(student_solutions[problemKey].keys()):
      #print(student_solutions[problemKey][userKey])
      src = student_solutions[problemKey][userKey]
      features = code_features(ast.parse(src))

      for item in ["imports", "statements", "expressions", "functions"]:
        for things in list(set(features[item])):
            if things in dic[item]:
              dic[item][things] +=1
            else:
              dic[item][things] = 1

    prob_count +=1
    user_count_dic[problemKey] = user_count+1
    problem_key_dic[problemKey] = dic
    master_dic = {'problemSkills':{},'userSkills':{}}

  for problemKey in student_solutions.keys():
    for userKey in student_solutions[problemKey].keys():
      src = student_solutions[problemKey][userKey]
      analysis = code_features(ast.parse(src))
      newUserFeatures = {
          "statements":{},
          "functions":{},
          "imports":{},
          "expressions":{}
        }
      master_dic["userSkills"][userKey] = master_dic["userSkills"].get(userKey, newUserFeatures)
      for key in analysis:
        if key in master_dic["userSkills"][userKey]:
          for things in analysis[key]:
            if things not in master_dic["userSkills"][userKey][key]:
              master_dic["userSkills"][userKey][key].update({things:{problemKey:True}})
            else:
              master_dic["userSkills"][userKey][key][things].update({problemKey:True})
        else:
          print("Why was key missing?", key)
      newUserFeatures = {
          "statements":{},
          "functions":{},
          "imports":{},
          "expressions":{}
        }
      master_dic["problemSkills"][problemKey] = master_dic["problemSkills"].get(problemKey, newUserFeatures)
      for key in analysis:
        if key in master_dic["problemSkills"][problemKey]:
          for features in analysis[key]:
            if features not in master_dic["problemSkills"][problemKey][key]:
              master_dic["problemSkills"][problemKey][key].update({features:{userKey:True}})
            else:
              master_dic["problemSkills"][problemKey][key][features].update({userKey:True})

  return master_dic

master_dic = getSolutionFeatures(student_solutions)

print(master_dic)


{'problemSkills': {'-LDkAuy-IFXOJoBILgL6': {'statements': {'Return': {'0NloChcpl6bV3cpfKk0zJx59IEZ2': True, '4dCdPX3MVlWNagub1pTpWBjNz2A2': True, '6Tkig5Bmg3SFm3WPjyY6JclHYO53': True, '85t8IgfUtld71Eobas8RvHCRZZk1': True, '95yMg7xQSIgYUdGDQXEcnn1gvJ83': True, 'Daa0PXSqy4OEc3jY45VuZwzn9JL2': True, 'InEAFwVEZNTqfg0RtCTdl79ME1p2': True, 'IrnPht6xFqcG0P436Gg0Ka8QX552': True, 'JaXyc6jxkhSNmd9LmYbi3SWVVT72': True, 'LlYmkwKZscTljV5ogObhEGBIjpi2': True, 'Oodwq4Vhh2T9zmgGkxd6kXsBZhw1': True, 'S3iSosxCuTPHGAHGPHSe4xrnjPG3': True, 'UhspAheoKvbMJ6rDLnBjKnMM4kj1': True, 'VVdCBPkgn2RascyF7E7KnNU3R5q1': True, 'WdDD6uW6icUhsnzntgPDSVUoCp42': True, 'Z84QafY8xCWefoOVfbEZah4UZiW2': True, 'cybE1DRNYtZCtCQrEYjZFI6oVa62': True, 'gqFNZRjR63UATiVX3omS0wQ2TzW2': True, 'hUK6CPhD4UWnKKSCBq3DU8OJo6O2': True, 'oTeidj1LEtcR9KrvTadnTGHDw9z2': True, 'ptF1d0QwSDOt2ApSkJa2tw3Aktn1': True, 'r168FElrvNhZlxwA9Fl3tPIhRze2': True, 'xHnJN4g1QKhrYsxNX3xjjl7AyF92': True, 'zbEJ9pSOHtUJwSO52qxt43bY62L2': True}}, 'functions': {},

Make a lambda function for all code above this point.




In [ ]:
for key in master_dic:
  print(key)

problemSkills
userSkills


In [ ]:
#printing out problemskills
userSkills = {}
problemSkills = {}

userSkills = master_dic["userSkills"]
problemSkills = master_dic["problemSkills"]

for x in problemSkills:
    print("")
    print (x)
    print("")
    for y in problemSkills[x]:
        print("")
        print (y)
        print("")
        for z in problemSkills[x][y]:
          print (z,':',problemSkills[x][y][z])


-LDkAuy-IFXOJoBILgL6


statements

Return : {'0NloChcpl6bV3cpfKk0zJx59IEZ2': True, '4dCdPX3MVlWNagub1pTpWBjNz2A2': True, '6Tkig5Bmg3SFm3WPjyY6JclHYO53': True, '85t8IgfUtld71Eobas8RvHCRZZk1': True, '95yMg7xQSIgYUdGDQXEcnn1gvJ83': True, 'Daa0PXSqy4OEc3jY45VuZwzn9JL2': True, 'InEAFwVEZNTqfg0RtCTdl79ME1p2': True, 'IrnPht6xFqcG0P436Gg0Ka8QX552': True, 'JaXyc6jxkhSNmd9LmYbi3SWVVT72': True, 'LlYmkwKZscTljV5ogObhEGBIjpi2': True, 'Oodwq4Vhh2T9zmgGkxd6kXsBZhw1': True, 'S3iSosxCuTPHGAHGPHSe4xrnjPG3': True, 'UhspAheoKvbMJ6rDLnBjKnMM4kj1': True, 'VVdCBPkgn2RascyF7E7KnNU3R5q1': True, 'WdDD6uW6icUhsnzntgPDSVUoCp42': True, 'Z84QafY8xCWefoOVfbEZah4UZiW2': True, 'cybE1DRNYtZCtCQrEYjZFI6oVa62': True, 'gqFNZRjR63UATiVX3omS0wQ2TzW2': True, 'hUK6CPhD4UWnKKSCBq3DU8OJo6O2': True, 'oTeidj1LEtcR9KrvTadnTGHDw9z2': True, 'ptF1d0QwSDOt2ApSkJa2tw3Aktn1': True, 'r168FElrvNhZlxwA9Fl3tPIhRze2': True, 'xHnJN4g1QKhrYsxNX3xjjl7AyF92': True, 'zbEJ9pSOHtUJwSO52qxt43bY62L2': True}

functions


imports


expressions

* : {'

In [ ]:
#printing out userSkills
for x in userSkills:
    print("")
    print (x)
    print("")
    for y in userSkills[x]:
        print("")
        print (y)
        print("")
        for z in userSkills[x][y]:
          print (z,':',userSkills[x][y][z])


0NloChcpl6bV3cpfKk0zJx59IEZ2


statements

Return : {'-LDkAuy-IFXOJoBILgL6': True, '-LDkAxmbeJ9r3DcfU0ZW': True, '-LDkB1-McVTufbnKUw-8': True, '-LDkB3z2Pp9wRf-zCxBk': True, '-LDkB7FprNZsNyK67zFC': True, '-LDkBAqpHP6trY1K4Bj5': True, '-LDkBE8kMmBdo9gKt02Z': True, '-LDkBHslDND-YNg-u9i1': True, '-LDkBNZ-WjGg-vAEHofN': True}
If : {'-LDkAxmbeJ9r3DcfU0ZW': True, '-LDkB1-McVTufbnKUw-8': True, '-LDkB7FprNZsNyK67zFC': True, '-LDkBAqpHP6trY1K4Bj5': True}
For : {'-LDkB1-McVTufbnKUw-8': True, '-LDkB7FprNZsNyK67zFC': True, '-LDkBAqpHP6trY1K4Bj5': True}

functions

sum : {'-LDkB3z2Pp9wRf-zCxBk': True}
type : {'-LDkB7FprNZsNyK67zFC': True}
.replace : {'-LDkBE8kMmBdo9gKt02Z': True}
dict : {'-LDkBHslDND-YNg-u9i1': True}
zip : {'-LDkBHslDND-YNg-u9i1': True}
.values : {'-LDkBHslDND-YNg-u9i1': True}
.keys : {'-LDkBHslDND-YNg-u9i1': True}
Counter : {'-LDkBNZ-WjGg-vAEHofN': True}

imports

Counter : {'-LDkBNZ-WjGg-vAEHofN': True}

expressions

* : {'-LDkAuy-IFXOJoBILgL6': True}
> : {'-LDkAxmbeJ9r3DcfU0ZW':

In [ ]:
#get the problem percentagees
def getProblemPercentiles(master_dic,student_solutions):
  #featureProblemPercentiles -> featureType -> feature -> problemKey -> percentage -> 0.76

  #get user count first
  user_count_dic = {}
  for problemKey in student_solutions:
    user_count = 0
    for userKey in student_solutions[problemKey]:
        user_count += 1

    user_count_dic[problemKey] = user_count
  print(user_count_dic)


  featureProblemPercentiles = {}
  for problemKey in master_dic["problemSkills"]:
    for featureType in master_dic["problemSkills"][problemKey]:
      for features in master_dic["problemSkills"][problemKey][featureType]:
        if featureType not in featureProblemPercentiles:
          featureProblemPercentiles[featureType] = {}
        if features not in featureProblemPercentiles[featureType]:
          featureProblemPercentiles[featureType][features] = {}
        if problemKey not in featureProblemPercentiles[featureType][features]:
          featureProblemPercentiles[featureType][features][problemKey] = 0

        feature_count = len(master_dic["problemSkills"][problemKey][featureType][features])

        user_count = user_count_dic[problemKey]

        percentage = feature_count/user_count*100

        featureProblemPercentiles[featureType][features][problemKey] = percentage
  return featureProblemPercentiles




featureProblemPercentiles = getProblemPercentiles(master_dic,student_solutions)
sorted_percentage = {}

for featureType in featureProblemPercentiles:
  for feature in featureProblemPercentiles[featureType]:
    for problemKey in featureProblemPercentiles[featureType][feature]:
      if featureType not in sorted_percentage:
        sorted_percentage[featureType] = {}
      if feature not in sorted_percentage[featureType]:
        sorted_percentage[featureType][feature] = []

      sorted_percentage[featureType][feature] = sorted(featureProblemPercentiles[featureType][feature].items(), key=lambda x: x[1],reverse=True)
    print(featureType)
    print("")
    print(feature)
    print("")
    print(sorted_percentage[featureType][feature])

print(featureProblemPercentiles)


{'-LDkAuy-IFXOJoBILgL6': 24, '-LDkAxmbeJ9r3DcfU0ZW': 18, '-LDkB1-McVTufbnKUw-8': 16, '-LDkB3z2Pp9wRf-zCxBk': 16, '-LDkB7FprNZsNyK67zFC': 16, '-LDkBAqpHP6trY1K4Bj5': 15, '-LDkBE8kMmBdo9gKt02Z': 13, '-LDkBHslDND-YNg-u9i1': 13, '-LDkBNZ-WjGg-vAEHofN': 12}
statements

Return

[('-LDkAuy-IFXOJoBILgL6', 100.0), ('-LDkAxmbeJ9r3DcfU0ZW', 100.0), ('-LDkB1-McVTufbnKUw-8', 100.0), ('-LDkB7FprNZsNyK67zFC', 100.0), ('-LDkBAqpHP6trY1K4Bj5', 100.0), ('-LDkBE8kMmBdo9gKt02Z', 100.0), ('-LDkBHslDND-YNg-u9i1', 100.0), ('-LDkB3z2Pp9wRf-zCxBk', 93.75), ('-LDkBNZ-WjGg-vAEHofN', 91.66666666666666)]
statements

If

[('-LDkAxmbeJ9r3DcfU0ZW', 100.0), ('-LDkB7FprNZsNyK67zFC', 75.0), ('-LDkB1-McVTufbnKUw-8', 68.75), ('-LDkBAqpHP6trY1K4Bj5', 66.66666666666666), ('-LDkBNZ-WjGg-vAEHofN', 50.0), ('-LDkBE8kMmBdo9gKt02Z', 15.384615384615385), ('-LDkB3z2Pp9wRf-zCxBk', 6.25)]
statements

For

[('-LDkB7FprNZsNyK67zFC', 75.0), ('-LDkBAqpHP6trY1K4Bj5', 73.33333333333333), ('-LDkB1-McVTufbnKUw-8', 68.75), ('-LDkBNZ-WjGg-vAEH

In [ ]:
#printing out featureProblemPercentiles

for x in featureProblemPercentiles:
    print("")
    print (x)
    print("")
    for y in featureProblemPercentiles[x]:
        print("")
        print (y)
        print("")
        for z in featureProblemPercentiles[x][y]:
          print (z,':',featureProblemPercentiles[x][y][z])


statements


Return

-LDkAuy-IFXOJoBILgL6 : 100.0
-LDkAxmbeJ9r3DcfU0ZW : 100.0
-LDkB1-McVTufbnKUw-8 : 100.0
-LDkB3z2Pp9wRf-zCxBk : 93.75
-LDkB7FprNZsNyK67zFC : 100.0
-LDkBAqpHP6trY1K4Bj5 : 100.0
-LDkBE8kMmBdo9gKt02Z : 100.0
-LDkBHslDND-YNg-u9i1 : 100.0
-LDkBNZ-WjGg-vAEHofN : 91.66666666666666

If

-LDkAxmbeJ9r3DcfU0ZW : 100.0
-LDkB1-McVTufbnKUw-8 : 68.75
-LDkB3z2Pp9wRf-zCxBk : 6.25
-LDkB7FprNZsNyK67zFC : 75.0
-LDkBAqpHP6trY1K4Bj5 : 66.66666666666666
-LDkBE8kMmBdo9gKt02Z : 15.384615384615385
-LDkBNZ-WjGg-vAEHofN : 50.0

For

-LDkB1-McVTufbnKUw-8 : 68.75
-LDkB3z2Pp9wRf-zCxBk : 56.25
-LDkB7FprNZsNyK67zFC : 75.0
-LDkBAqpHP6trY1K4Bj5 : 73.33333333333333
-LDkBE8kMmBdo9gKt02Z : 7.6923076923076925
-LDkBHslDND-YNg-u9i1 : 46.15384615384615
-LDkBNZ-WjGg-vAEHofN : 66.66666666666666

While

-LDkBE8kMmBdo9gKt02Z : 7.6923076923076925
-LDkBHslDND-YNg-u9i1 : 7.6923076923076925

expressions


*

-LDkAuy-IFXOJoBILgL6 : 100.0
-LDkAxmbeJ9r3DcfU0ZW : 5.555555555555555

>

-LDkAxmbeJ9r3DcfU0ZW : 88.88888888

In [ ]:
def getFeatureRanking(master_dic):
  #featureRanking -> {“featureType”:a, “feature”:b, “userCount”:c}
  #userSkills-> UserKey -> featureType -> feature -> problemKey -> True

  content = []
  #get userCount
  usercount = {}
  for userKey in master_dic["userSkills"]:
    for featureType in master_dic["userSkills"][userKey]:
      for features in master_dic["userSkills"][userKey][featureType]:
        if features not in usercount:
          usercount.update({features:1})
        else:
           usercount[features] = usercount[features]+1

  print(usercount)




  for userKey in master_dic["userSkills"]:
    for featureType in master_dic["userSkills"][userKey]:
      for features in master_dic["userSkills"][userKey][featureType]:

        featureRanking = {}
        featureRanking["featureType"] = featureType
        featureRanking["feature"] = features
        featureRanking["userCount"] = usercount[features]
        content.append(featureRanking)

  final_ranking = []
  for things in content:
    if things not in final_ranking:
      final_ranking.append(things)

  return sorted(final_ranking, key=lambda k: k['userCount'],reverse=True)


featureRanking = getFeatureRanking(master_dic)
print(featureRanking)

print(" ")
for things in featureRanking:
  print(" ")
  print(things)
  print(" ")



{'Return': 24, 'If': 18, 'For': 14, 'sum': 7, 'type': 8, '.replace': 11, 'dict': 7, 'zip': 2, '.values': 4, '.keys': 5, 'Counter': 6, '*': 24, '>': 16, '==': 17, '+': 14, 'len': 8, '<': 9, 'isinstance': 8, 'tuple': 1, '.items': 7, 'list': 3, '.count': 1, 'histogram': 1, 'While': 2, 'int': 2, 'eval': 1, 'str': 1, 'map': 1, '.find': 1, 'range': 1, '-': 1, 'filter': 1, 'defaultdict': 2}
[{'featureType': 'statements', 'feature': 'Return', 'userCount': 24}, {'featureType': 'expressions', 'feature': '*', 'userCount': 24}, {'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}, {'featureType': 'expressions', 'feature': '>', 'userCount': 16}, {'featureType': 'statements', 'feature': 'For', 'userCount': 14}, {'featureType': 'expressions', 'feature': '+', 'userCount': 14}, {'featureType': 'functions', 'feature': '.replace', 'userCount': 11}, {'featureType': 'expressions', 'feature': '<', 'userCount': 9}, {'featureType': '

In [ ]:
def getMissingFeatures(master_dic, featureRanking):
  recommendations = {}
  for things in featureRanking:
    for userKey in master_dic['userSkills']:
      flag = False
      for featureType in master_dic['userSkills'][userKey]:
          if things['featureType'] == featureType:
            if things['feature'] in master_dic['userSkills'][userKey][featureType]:
              flag = True
      if not flag:
        if userKey not in recommendations:
          recommendations.update({userKey:[things]})
        else:
          recommendations[userKey].append(things)


  return recommendations

recos = getMissingFeatures(master_dic,featureRanking)
for userKey in recos:
    recos[userKey] = recos[userKey][:2]

print(recos)
for userKey in recos:
  print("")
  print(userKey)
  print("")
  print(recos[userKey])




{'85t8IgfUtld71Eobas8RvHCRZZk1': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], '95yMg7xQSIgYUdGDQXEcnn1gvJ83': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], 'InEAFwVEZNTqfg0RtCTdl79ME1p2': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], 'IrnPht6xFqcG0P436Gg0Ka8QX552': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], 'JaXyc6jxkhSNmd9LmYbi3SWVVT72': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], 'S3iSosxCuTPHGAHGPHSe4xrnjPG3': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17}], 'UhspAheo

In [ ]:
  '''nextRecommendations -> userKey -> {
“lastUpdated”: TIMESTAMP,
“problems”: [
 { “featureType”:aa, “feature”: a, “likelihood”:76,'problemKey':},
{ “featureType”:bb, “feature“:b, likelihood”:70,'problemKey':},
{ “featureType”:cc, “feature”:c, “likelihood”:60,'problemKey':},
]'''
#featureProblemPercentiles -> featureType -> feature -> problemKey -> percentage -> 0.76
def userProblem(master_dic):
    userProblemKey = {}
    for userKey in master_dic['userSkills']:
      for featureType in master_dic['userSkills'][userKey]:
        for feature in master_dic['userSkills'][userKey][featureType]:
          for problemKey in master_dic['userSkills'][userKey][featureType][feature]:
            if userKey not in userProblemKey:
              userProblemKey[userKey] = []
            if problemKey not in userProblemKey[userKey]:
              userProblemKey[userKey].append(problemKey)
    return userProblemKey

userProblemKey = userProblem(master_dic)



print(" ")
print(" ")

def getProblemRecos(recos,sorted_percentage):
    nextRecommendations = {}

    for userKey in recos:
      if userKey not in nextRecommendations:
        nextRecommendations[userKey] = {}

      nextRecommendations[userKey]['lastUpdated'] = ''

      nextRecommendations[userKey]['problems'] = []
      for things in recos[userKey]:
        nextRecommendations[userKey]['problems'].append(dict(things))

      for index in range(len(nextRecommendations[userKey]['problems'])):

        nextRecommendations[userKey]['problems'][index]['solvedFeatureProblems'] = {}
        nextRecommendations[userKey]['problems'][index]['percentage'] = 0
        nextRecommendations[userKey]['problems'][index]['problem'] = 'null'
        feature = nextRecommendations[userKey]['problems'][index]['feature']
        featureType = nextRecommendations[userKey]['problems'][index]['featureType']

        for count in range(len(sorted_percentage[featureType][feature])):
          if sorted_percentage[featureType][feature][count][0] in userProblemKey[userKey]:

            if sorted_percentage[featureType][feature][count][0] not in nextRecommendations[userKey]['problems'][index]['solvedFeatureProblems']:

              nextRecommendations[userKey]['problems'][index]['solvedFeatureProblems'].update({sorted_percentage[featureType][feature][count][0]:
                                                                                              sorted_percentage[featureType][feature][count][1]})

          else:
            nextRecommendations[userKey]['problems'][index]['percentage'] = sorted_percentage[featureType][feature][count][1]
            nextRecommendations[userKey]['problems'][index]['problem'] = sorted_percentage[featureType][feature][count][0]
            break




    return nextRecommendations


nextRecommendations = getProblemRecos(recos,sorted_percentage)
print(nextRecommendations)



 
 
{'85t8IgfUtld71Eobas8RvHCRZZk1': {'lastUpdated': '', 'problems': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18, 'solvedFeatureProblems': {}, 'percentage': 100.0, 'problem': '-LDkAxmbeJ9r3DcfU0ZW'}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}]}, '95yMg7xQSIgYUdGDQXEcnn1gvJ83': {'lastUpdated': '', 'problems': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18, 'solvedFeatureProblems': {}, 'percentage': 100.0, 'problem': '-LDkAxmbeJ9r3DcfU0ZW'}, {'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}]}, 'InEAFwVEZNTqfg0RtCTdl79ME1p2': {'lastUpdated': '', 'problems': [{'featureType': 'statements', 'feature': 'If', 'userCount': 18, 'solvedFeatureProblems': {}, 'percentage': 100.0, 'problem': '-LDkAxmbeJ9r3DcfU0ZW'}, {'featureType': 'expressions', 'feature': '=

In [ ]:


for x in nextRecommendations:
    print("")
    print (x)
    print("")
    for y in nextRecommendations[x]:
        print("")
        print (y)
        print("")
        for z in nextRecommendations[x][y]:
          for things in z:
            print("")
            print(things,z[things])
            print("")



85t8IgfUtld71Eobas8RvHCRZZk1


lastUpdated


problems


featureType statements


feature If


userCount 18


solvedFeatureProblems {}


percentage 100.0


problem -LDkAxmbeJ9r3DcfU0ZW


featureType expressions


feature ==


userCount 17


solvedFeatureProblems {}


percentage 93.75


problem -LDkB1-McVTufbnKUw-8


95yMg7xQSIgYUdGDQXEcnn1gvJ83


lastUpdated


problems


featureType statements


feature If


userCount 18


solvedFeatureProblems {}


percentage 100.0


problem -LDkAxmbeJ9r3DcfU0ZW


featureType expressions


feature ==


userCount 17


solvedFeatureProblems {}


percentage 93.75


problem -LDkB1-McVTufbnKUw-8


InEAFwVEZNTqfg0RtCTdl79ME1p2


lastUpdated


problems


featureType statements


feature If


userCount 18


solvedFeatureProblems {}


percentage 100.0


problem -LDkAxmbeJ9r3DcfU0ZW


featureType expressions


feature ==


userCount 17


solvedFeatureProblems {}


percentage 93.75


problem -LDkB1-McVTufbnKUw-8


IrnPht6xFqcG0P436Gg0Ka8QX552


lastUpdated


pro

In [ ]:
########################## FOR FPP #####################################
fpp = {}
for i in featureProblemPercentiles:
  for feat in featureProblemPercentiles[i]:
    if feat[0] == '.':
      new_feat = '-'+feat[1:]
      fpp[new_feat+"_"+i] = {}
    else:
      fpp[feat+"_"+i] = {}
    for qns in featureProblemPercentiles[i][feat]:
      # can just use new_feat here but only when i make sure feat originally started with '.'
      if feat[0] == ".":
        fpp[new_feat+"_"+i][qns] = {}
        fpp[new_feat+"_"+i][qns]["feature"] = new_feat
        fpp[new_feat+"_"+i][qns]["featureType"] = i
        fpp[new_feat+"_"+i][qns]["percentage"] = featureProblemPercentiles[i][feat][qns]

      else:
        fpp[feat+"_"+i][qns] = {}
        fpp[feat+"_"+i][qns]["feature"] = feat
        fpp[feat+"_"+i][qns]["featureType"] = i
        fpp[feat+"_"+i][qns]["percentage"] = featureProblemPercentiles[i][feat][qns]

print(fpp)

{'Return_statements': {'-LDkAuy-IFXOJoBILgL6': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkAxmbeJ9r3DcfU0ZW': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkB1-McVTufbnKUw-8': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkB3z2Pp9wRf-zCxBk': {'feature': 'Return', 'featureType': 'statements', 'percentage': 93.75}, '-LDkB7FprNZsNyK67zFC': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkBAqpHP6trY1K4Bj5': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkBE8kMmBdo9gKt02Z': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkBHslDND-YNg-u9i1': {'feature': 'Return', 'featureType': 'statements', 'percentage': 100.0}, '-LDkBNZ-WjGg-vAEHofN': {'feature': 'Return', 'featureType': 'statements', 'percentage': 91.66666666666666}}, 'If_statements': {'-LDkAxmbeJ9r3DcfU0ZW': {'feature': 'If', 'featureType': 'statements', 'p

In [ ]:
############################## FOR FEATURERANKING ###############################

## Change . to -
for item in featureRanking:
  for feat in item['feature']:
    if feat[0] == '.':
      new_feat = '-'+feat[1:]
      item['feature'] = new_feat

## change to {feature_featureType: usercount})
featRank = {}
for item in featureRanking:
  newkey = item["feature"]+'_'+item["featureType"]
  featRank[newkey] = {}
  featRank[newkey]["feature"] = item["feature"]
  featRank[newkey]["featureType"] = item["featureType"]
  featRank[newkey]["userCount"] = item["userCount"]

print(featRank)

{'Return_statements': {'feature': 'Return', 'featureType': 'statements', 'userCount': 24}, '*_expressions': {'feature': '*', 'featureType': 'expressions', 'userCount': 24}, 'If_statements': {'feature': 'If', 'featureType': 'statements', 'userCount': 18}, '==_expressions': {'feature': '==', 'featureType': 'expressions', 'userCount': 17}, '>_expressions': {'feature': '>', 'featureType': 'expressions', 'userCount': 16}, 'For_statements': {'feature': 'For', 'featureType': 'statements', 'userCount': 14}, '+_expressions': {'feature': '+', 'featureType': 'expressions', 'userCount': 14}, '-_functions': {'feature': '-', 'featureType': 'functions', 'userCount': 1}, '<_expressions': {'feature': '<', 'featureType': 'expressions', 'userCount': 9}, 'type_functions': {'feature': 'type', 'featureType': 'functions', 'userCount': 8}, 'len_functions': {'feature': 'len', 'featureType': 'functions', 'userCount': 8}, 'isinstance_functions': {'feature': 'isinstance', 'featureType': 'functions', 'userCount': 

In [ ]:
######################### FOR NEXTRECOMMENDATIONS ###############################


userReco = {}

for user in nextRecommendations:
  userReco[user] = {}
  userReco[user]['lastUpdated'] = nextRecommendations[user]['lastUpdated']
  userReco[user]['rankOrder'] = {}
  for feature in nextRecommendations[user]['problems']:
    userReco[user]['problems'] = {}
    probkey = feature['problem']

    if feature['feature'][0] == '.':
      new_feat = '-' + feature['feature'][1:] + '_' + feature['featureType']
      userReco[user]['problems'][new_feat] = feature

    else:
      new_feat = feature['feature'] + '_' + feature['featureType']
      userReco[user]['problems'][new_feat] = feature



for userKey in userReco:
  print(userKey)
  print("")
  for things in userReco[userKey]:
    print(things)
    print("")
    for stuff in userReco[userKey][things]:
      print(stuff)
      print("")
      print(userReco[userKey][things][stuff])
      print("")

85t8IgfUtld71Eobas8RvHCRZZk1

lastUpdated

rankOrder

problems

==_expressions

{'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}

95yMg7xQSIgYUdGDQXEcnn1gvJ83

lastUpdated

rankOrder

problems

==_expressions

{'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}

InEAFwVEZNTqfg0RtCTdl79ME1p2

lastUpdated

rankOrder

problems

==_expressions

{'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}

IrnPht6xFqcG0P436Gg0Ka8QX552

lastUpdated

rankOrder

problems

==_expressions

{'featureType': 'expressions', 'feature': '==', 'userCount': 17, 'solvedFeatureProblems': {}, 'percentage': 93.75, 'problem': '-LDkB1-McVTufbnKUw-8'}

JaXyc6jxkhSNmd9LmYbi3SWVVT72

lastUpdated

rankOrder

problems

==_expressio